In [1]:
%matplotlib inline
import numpy as np
import os, glob
import matplotlib.pyplot as plt
from netCDF4 import Dataset

import timeit
from pandas import to_datetime, date_range
import wrf, xarray

In [2]:
plt.rc("font",size=14)

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### define paths

In [4]:
dataPath = '/glade/scratch/domingom/Cheyenne/XPIA_mesoLES/SIMULS/'
outPath  = '/glade/scratch/doubrawa/postProcessing/'

### define some parameters

In [5]:
n_processors = 1800
domainId     = 4
prefix       = "LES_25m" if domainId==4 else "LES_100m"

In [6]:
nt_per_file        = 180 if domainId==4 else 30
nhours             = 10
nfiles             = nhours * 2
nt_total           = nt_per_file * nfiles

n_east_west        = 1200
n_east_west_stag   = n_east_west + 1
n_south_north      = 1200
n_south_north_stag = n_south_north + 1

## open all data

In [7]:
year           = 2018
month          = 3
day            = 21
hour           = 18
minute         = 0
half_hour_idx  = 1 if minute<30 else 2
datetime       = to_datetime("{0}-{1}-{2} {3}:{4}".format(year,month,day,hour,minute),format="%Y-%m-%d %H:%M")
print(datetime)

2018-03-21 18:00:00


In [8]:
variables_2d = ["XLAT","XLONG","ter","RMOL","HFX"]
variables_3d = ["U","V","wa","theta","z"]
variables    = variables_2d + variables_3d
data         = {}
for var in ["U"]:
    data[var] = np.zeros((72,n_south_north,n_east_west_stag))
for var in ["V"]:
    data[var] = np.zeros((72,n_south_north_stag,n_east_west))    
for var in variables_2d:
    data[var] = np.zeros((1200,1200))
for var in ["wa","theta","z"]:
    data[var] = np.zeros((72,n_south_north,n_east_west))        

In [9]:
for processor in range(n_processors):

    wrfoutPath     = (glob.glob(os.path.join(dataPath,
                               'WRF_mesoLES_4dom_RAP_2015_03_{0}_mesoLES'.format(day-1),
                               'HOUR_{0}_{1}'.format(hour,half_hour_idx),
                               'wrfout_d0{0}_*_{1:04d}'.format(domainId,processor))))[0]
    print (os.path.split(wrfoutPath)[-1])
    wrfnc = xarray.open_dataset(wrfoutPath)
    wrfnc_for_wrf = Dataset(wrfoutPath,'r')
    wrf_datetime = to_datetime(wrfnc.XTIME[0].data)


    if ((wrf_datetime-datetime).seconds > 5 * 60.0):
        print ("ERROR!!!")   
    
    #
    # get all variables (3-d in space) 
    #
    for var in variables:

        try:
            data_tmp = wrf.getvar(wrfnc_for_wrf, var, timeidx=0).data
        except:
            data_tmp = wrfnc[var].isel(Time=0).data       
            
        we_0 = getattr(wrfnc,'WEST-EAST_PATCH_START_UNSTAG') - 1        
        we_1 = getattr(wrfnc,'WEST-EAST_PATCH_END_UNSTAG')                

        sn_0 = getattr(wrfnc,'SOUTH-NORTH_PATCH_START_UNSTAG') - 1       
        sn_1 = getattr(wrfnc,'SOUTH-NORTH_PATCH_END_UNSTAG')               
            
            
            
        if data_tmp.ndim==3:
            if var=='U':
                we_0 = getattr(wrfnc,'WEST-EAST_PATCH_START_STAG') - 1        
                we_1 = getattr(wrfnc,'WEST-EAST_PATCH_END_STAG')                                
            if var=='V':
                sn_0 = getattr(wrfnc,'SOUTH-NORTH_PATCH_START_STAG') - 1       
                sn_1 = getattr(wrfnc,'SOUTH-NORTH_PATCH_END_STAG')                                                                       
            data[var][:, sn_0:sn_1, we_0:we_1] = data_tmp.copy()
        else:
            data[var][sn_0:sn_1, we_0:we_1] = data_tmp.copy()            

wrfout_d04_2015-03-21_18:00:10_0000
wrfout_d04_2015-03-21_18:00:10_0001
wrfout_d04_2015-03-21_18:00:10_0002
wrfout_d04_2015-03-21_18:00:10_0003
wrfout_d04_2015-03-21_18:00:10_0004
wrfout_d04_2015-03-21_18:00:10_0005
wrfout_d04_2015-03-21_18:00:10_0006
wrfout_d04_2015-03-21_18:00:10_0007
wrfout_d04_2015-03-21_18:00:10_0008
wrfout_d04_2015-03-21_18:00:10_0009
wrfout_d04_2015-03-21_18:00:10_0010
wrfout_d04_2015-03-21_18:00:10_0011
wrfout_d04_2015-03-21_18:00:10_0012
wrfout_d04_2015-03-21_18:00:10_0013
wrfout_d04_2015-03-21_18:00:10_0014
wrfout_d04_2015-03-21_18:00:10_0015
wrfout_d04_2015-03-21_18:00:10_0016
wrfout_d04_2015-03-21_18:00:10_0017
wrfout_d04_2015-03-21_18:00:10_0018
wrfout_d04_2015-03-21_18:00:10_0019
wrfout_d04_2015-03-21_18:00:10_0020
wrfout_d04_2015-03-21_18:00:10_0021
wrfout_d04_2015-03-21_18:00:10_0022
wrfout_d04_2015-03-21_18:00:10_0023
wrfout_d04_2015-03-21_18:00:10_0024
wrfout_d04_2015-03-21_18:00:10_0025
wrfout_d04_2015-03-21_18:00:10_0026
wrfout_d04_2015-03-21_18:00:

wrfout_d04_2015-03-21_18:00:10_0228
wrfout_d04_2015-03-21_18:00:10_0229
wrfout_d04_2015-03-21_18:00:10_0230
wrfout_d04_2015-03-21_18:00:10_0231
wrfout_d04_2015-03-21_18:00:10_0232
wrfout_d04_2015-03-21_18:00:10_0233
wrfout_d04_2015-03-21_18:00:10_0234
wrfout_d04_2015-03-21_18:00:10_0235
wrfout_d04_2015-03-21_18:00:10_0236
wrfout_d04_2015-03-21_18:00:10_0237
wrfout_d04_2015-03-21_18:00:10_0238
wrfout_d04_2015-03-21_18:00:10_0239
wrfout_d04_2015-03-21_18:00:10_0240
wrfout_d04_2015-03-21_18:00:10_0241
wrfout_d04_2015-03-21_18:00:10_0242
wrfout_d04_2015-03-21_18:00:10_0243
wrfout_d04_2015-03-21_18:00:10_0244
wrfout_d04_2015-03-21_18:00:10_0245
wrfout_d04_2015-03-21_18:00:10_0246
wrfout_d04_2015-03-21_18:00:10_0247
wrfout_d04_2015-03-21_18:00:10_0248
wrfout_d04_2015-03-21_18:00:10_0249
wrfout_d04_2015-03-21_18:00:10_0250
wrfout_d04_2015-03-21_18:00:10_0251
wrfout_d04_2015-03-21_18:00:10_0252
wrfout_d04_2015-03-21_18:00:10_0253
wrfout_d04_2015-03-21_18:00:10_0254
wrfout_d04_2015-03-21_18:00:

wrfout_d04_2015-03-21_18:00:10_0456
wrfout_d04_2015-03-21_18:00:10_0457
wrfout_d04_2015-03-21_18:00:10_0458
wrfout_d04_2015-03-21_18:00:10_0459
wrfout_d04_2015-03-21_18:00:10_0460
wrfout_d04_2015-03-21_18:00:10_0461
wrfout_d04_2015-03-21_18:00:10_0462
wrfout_d04_2015-03-21_18:00:10_0463
wrfout_d04_2015-03-21_18:00:10_0464
wrfout_d04_2015-03-21_18:00:10_0465
wrfout_d04_2015-03-21_18:00:10_0466
wrfout_d04_2015-03-21_18:00:10_0467
wrfout_d04_2015-03-21_18:00:10_0468
wrfout_d04_2015-03-21_18:00:10_0469
wrfout_d04_2015-03-21_18:00:10_0470
wrfout_d04_2015-03-21_18:00:10_0471
wrfout_d04_2015-03-21_18:00:10_0472
wrfout_d04_2015-03-21_18:00:10_0473
wrfout_d04_2015-03-21_18:00:10_0474
wrfout_d04_2015-03-21_18:00:10_0475
wrfout_d04_2015-03-21_18:00:10_0476
wrfout_d04_2015-03-21_18:00:10_0477
wrfout_d04_2015-03-21_18:00:10_0478
wrfout_d04_2015-03-21_18:00:10_0479
wrfout_d04_2015-03-21_18:00:10_0480
wrfout_d04_2015-03-21_18:00:10_0481
wrfout_d04_2015-03-21_18:00:10_0482
wrfout_d04_2015-03-21_18:00:

wrfout_d04_2015-03-21_18:00:10_0684
wrfout_d04_2015-03-21_18:00:10_0685
wrfout_d04_2015-03-21_18:00:10_0686
wrfout_d04_2015-03-21_18:00:10_0687
wrfout_d04_2015-03-21_18:00:10_0688
wrfout_d04_2015-03-21_18:00:10_0689
wrfout_d04_2015-03-21_18:00:10_0690
wrfout_d04_2015-03-21_18:00:10_0691
wrfout_d04_2015-03-21_18:00:10_0692
wrfout_d04_2015-03-21_18:00:10_0693
wrfout_d04_2015-03-21_18:00:10_0694
wrfout_d04_2015-03-21_18:00:10_0695
wrfout_d04_2015-03-21_18:00:10_0696
wrfout_d04_2015-03-21_18:00:10_0697
wrfout_d04_2015-03-21_18:00:10_0698
wrfout_d04_2015-03-21_18:00:10_0699
wrfout_d04_2015-03-21_18:00:10_0700
wrfout_d04_2015-03-21_18:00:10_0701
wrfout_d04_2015-03-21_18:00:10_0702
wrfout_d04_2015-03-21_18:00:10_0703
wrfout_d04_2015-03-21_18:00:10_0704
wrfout_d04_2015-03-21_18:00:10_0705
wrfout_d04_2015-03-21_18:00:10_0706
wrfout_d04_2015-03-21_18:00:10_0707
wrfout_d04_2015-03-21_18:00:10_0708
wrfout_d04_2015-03-21_18:00:10_0709
wrfout_d04_2015-03-21_18:00:10_0710
wrfout_d04_2015-03-21_18:00:

wrfout_d04_2015-03-21_18:00:10_0912
wrfout_d04_2015-03-21_18:00:10_0913
wrfout_d04_2015-03-21_18:00:10_0914
wrfout_d04_2015-03-21_18:00:10_0915
wrfout_d04_2015-03-21_18:00:10_0916
wrfout_d04_2015-03-21_18:00:10_0917
wrfout_d04_2015-03-21_18:00:10_0918
wrfout_d04_2015-03-21_18:00:10_0919
wrfout_d04_2015-03-21_18:00:10_0920
wrfout_d04_2015-03-21_18:00:10_0921
wrfout_d04_2015-03-21_18:00:10_0922
wrfout_d04_2015-03-21_18:00:10_0923
wrfout_d04_2015-03-21_18:00:10_0924
wrfout_d04_2015-03-21_18:00:10_0925
wrfout_d04_2015-03-21_18:00:10_0926
wrfout_d04_2015-03-21_18:00:10_0927
wrfout_d04_2015-03-21_18:00:10_0928
wrfout_d04_2015-03-21_18:00:10_0929
wrfout_d04_2015-03-21_18:00:10_0930
wrfout_d04_2015-03-21_18:00:10_0931
wrfout_d04_2015-03-21_18:00:10_0932
wrfout_d04_2015-03-21_18:00:10_0933
wrfout_d04_2015-03-21_18:00:10_0934
wrfout_d04_2015-03-21_18:00:10_0935
wrfout_d04_2015-03-21_18:00:10_0936
wrfout_d04_2015-03-21_18:00:10_0937
wrfout_d04_2015-03-21_18:00:10_0938
wrfout_d04_2015-03-21_18:00:

wrfout_d04_2015-03-21_18:00:10_1140
wrfout_d04_2015-03-21_18:00:10_1141
wrfout_d04_2015-03-21_18:00:10_1142
wrfout_d04_2015-03-21_18:00:10_1143
wrfout_d04_2015-03-21_18:00:10_1144
wrfout_d04_2015-03-21_18:00:10_1145
wrfout_d04_2015-03-21_18:00:10_1146
wrfout_d04_2015-03-21_18:00:10_1147
wrfout_d04_2015-03-21_18:00:10_1148
wrfout_d04_2015-03-21_18:00:10_1149
wrfout_d04_2015-03-21_18:00:10_1150
wrfout_d04_2015-03-21_18:00:10_1151
wrfout_d04_2015-03-21_18:00:10_1152
wrfout_d04_2015-03-21_18:00:10_1153
wrfout_d04_2015-03-21_18:00:10_1154
wrfout_d04_2015-03-21_18:00:10_1155
wrfout_d04_2015-03-21_18:00:10_1156
wrfout_d04_2015-03-21_18:00:10_1157
wrfout_d04_2015-03-21_18:00:10_1158
wrfout_d04_2015-03-21_18:00:10_1159
wrfout_d04_2015-03-21_18:00:10_1160
wrfout_d04_2015-03-21_18:00:10_1161
wrfout_d04_2015-03-21_18:00:10_1162
wrfout_d04_2015-03-21_18:00:10_1163
wrfout_d04_2015-03-21_18:00:10_1164
wrfout_d04_2015-03-21_18:00:10_1165
wrfout_d04_2015-03-21_18:00:10_1166
wrfout_d04_2015-03-21_18:00:

wrfout_d04_2015-03-21_18:00:10_1368
wrfout_d04_2015-03-21_18:00:10_1369
wrfout_d04_2015-03-21_18:00:10_1370
wrfout_d04_2015-03-21_18:00:10_1371
wrfout_d04_2015-03-21_18:00:10_1372
wrfout_d04_2015-03-21_18:00:10_1373
wrfout_d04_2015-03-21_18:00:10_1374
wrfout_d04_2015-03-21_18:00:10_1375
wrfout_d04_2015-03-21_18:00:10_1376
wrfout_d04_2015-03-21_18:00:10_1377
wrfout_d04_2015-03-21_18:00:10_1378
wrfout_d04_2015-03-21_18:00:10_1379
wrfout_d04_2015-03-21_18:00:10_1380
wrfout_d04_2015-03-21_18:00:10_1381
wrfout_d04_2015-03-21_18:00:10_1382
wrfout_d04_2015-03-21_18:00:10_1383
wrfout_d04_2015-03-21_18:00:10_1384
wrfout_d04_2015-03-21_18:00:10_1385
wrfout_d04_2015-03-21_18:00:10_1386
wrfout_d04_2015-03-21_18:00:10_1387
wrfout_d04_2015-03-21_18:00:10_1388
wrfout_d04_2015-03-21_18:00:10_1389
wrfout_d04_2015-03-21_18:00:10_1390
wrfout_d04_2015-03-21_18:00:10_1391
wrfout_d04_2015-03-21_18:00:10_1392
wrfout_d04_2015-03-21_18:00:10_1393
wrfout_d04_2015-03-21_18:00:10_1394
wrfout_d04_2015-03-21_18:00:

wrfout_d04_2015-03-21_18:00:10_1596
wrfout_d04_2015-03-21_18:00:10_1597
wrfout_d04_2015-03-21_18:00:10_1598
wrfout_d04_2015-03-21_18:00:10_1599
wrfout_d04_2015-03-21_18:00:10_1600
wrfout_d04_2015-03-21_18:00:10_1601
wrfout_d04_2015-03-21_18:00:10_1602
wrfout_d04_2015-03-21_18:00:10_1603
wrfout_d04_2015-03-21_18:00:10_1604
wrfout_d04_2015-03-21_18:00:10_1605
wrfout_d04_2015-03-21_18:00:10_1606
wrfout_d04_2015-03-21_18:00:10_1607
wrfout_d04_2015-03-21_18:00:10_1608
wrfout_d04_2015-03-21_18:00:10_1609
wrfout_d04_2015-03-21_18:00:10_1610
wrfout_d04_2015-03-21_18:00:10_1611
wrfout_d04_2015-03-21_18:00:10_1612
wrfout_d04_2015-03-21_18:00:10_1613
wrfout_d04_2015-03-21_18:00:10_1614
wrfout_d04_2015-03-21_18:00:10_1615
wrfout_d04_2015-03-21_18:00:10_1616
wrfout_d04_2015-03-21_18:00:10_1617
wrfout_d04_2015-03-21_18:00:10_1618
wrfout_d04_2015-03-21_18:00:10_1619
wrfout_d04_2015-03-21_18:00:10_1620
wrfout_d04_2015-03-21_18:00:10_1621
wrfout_d04_2015-03-21_18:00:10_1622
wrfout_d04_2015-03-21_18:00:

In [10]:
data["z"] = data["z"] - data["ter"]

# unstagger u and v

In [11]:
data["U"] = 0.5*(data["U"][:,:,0:n_east_west_stag-1] + data["U"][:,:,1:n_east_west_stag+1])

In [12]:
data["V"] = 0.5*(data["V"][:,0:n_south_north_stag-1,:] + data["V"][:,1:n_south_north_stag+1,:])

# get x,y mean

In [13]:
data_mean = {}
for var in ["U","V","wa","theta","z"]:
    data_mean[var] = np.mean(data[var],axis=(1,2))

# get x,y perturbations

In [ ]:
data_prime  = {}
for var in data_mean.keys():
    print (var)
    data_prime[var] = data[var] - data_mean[var][:,None,None]   

U
V
wa
theta


# compute fluxes

In [ ]:
fluxes = ["U_U","V_V","wa_wa","U_wa","V_wa","U_V","wa_theta"]
data_fluxes = {}
for flux in fluxes:
    var1 = flux.split("_")[0]
    var2 = flux.split("_")[1]
    data_fluxes[flux] = np.mean(data_prime[var1]*data_prime[var2],axis=(1,2))

In [ ]:
data_fluxes["wa_theta0"] = np.mean(data_prime["wa"]*data_prime["theta"][0,:,:],axis=(1,2))

# take a peek

In [ ]:
z_profile = data_mean["z"].copy()

In [ ]:
wth_profile = data_fluxes["wa_theta"].copy()

In [ ]:
fig = plt.figure(figsize=(6,4))
ax  = fig.add_subplot(111)

idx = np.where(z_profile<5000)[0]
idx_zi = np.argmin(wth_profile[idx])
zi = z_profile[idx_zi]

ax.plot(wth_profile,z_profile)
ax.set_ylim([0,5000])
ax.axhline(y=zi,color='gray',linestyle=":")
ax.axvline(x=0,color='gray',linestyle=":")

# save

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data_mean).set_index("z")

In [ ]:
df["z_std_xy"] = np.std(data["z"],axis=(1,2))

In [ ]:
df2 = pd.DataFrame(data_fluxes).set_index(df.index)

In [ ]:
df = pd.concat([df,df2],axis=1)

In [ ]:
column_mapping = {"U":"u",
 "V":"v",
 "wa":"w",
 "theta":"theta",
 "z_std_xy":"z_std_xy",
 "U_U":"u_u",
 "V_V":"v_v",
 "wa_wa":"w_w",
 "U_wa":"u_w",
 "V_wa":"v_w",
 "U_V":"u_v",
 "wa_theta":"w_theta",
 "wa_theta0":"w_theta0"}

In [ ]:
df.columns = [ column_mapping[col_old] for col_old in df.columns ]

In [ ]:
df.head()

In [ ]:
df.to_csv(os.path.join(outPath,"WRF_LES_25m_SPATIAL_AVERAGED_PROFILES_{0:%Y-%m-%d_%H:%M:%S}.csv".format(datetime)))